# Section 1: Business Understanding
    This is project about power usage data analysis. the data is real data of a Tranformer Area. The enegry data is measured by smart electronic meters (Meter). The data from meter is collected by a Terminal via PLC (Power Line Control Technology) one time per day include then report to server. It will be saved in the database in the database server:
    - Voltage U(V),
    - Current I(A),
    - Forward Active Energy A+(KWh)
    - Forward ReActive Energy R+(KWh)
    - Reverse Active Energy A-(KWh)
    - Reverse ReActive Energy R-(KWh)
    With multi tariff meter it have more data of each tariff :
    - Forward Active Energy tariff 1 A1+(KWh)
    - Forward Active Energy tariff 2 A2+(KWh)
    - Forward Active Energy tariff 3 A3+(KWh)
    - Forward Active Energy tariff 4 A3+(KWh)
    - Forward ReActive Energy tariff 1 R1+(Kvarh)
    - Forward ReActive Energy tariff 2 R2+(Kvarh)
    - Forward ReActive Energy tariff 3 R3+(Kvarh)
    - Forward ReActive Energy tariff 4 R3+(Kvarh)
    - Reverse Active Energy tariff 1 A1-(KWh)
    - Reverse Active Energy tariff 2 A2-(KWh)
    - Reverse Active Energy tariff 3 A3-(KWh)
    - Reverse Active Energy tariff 4 A3-(KWh)
    - Reverse ReActive Energy tariff 1 R1-(Kvarh)
    - Reverse ReActive Energy tariff 2 R2-(Kvarh)
    - Reverse ReActive Energy tariff 3 R3-(Kvarh)
    - Reverse ReActive Energy tariff 4 R3-(Kvarh)
    With this project we will know how about the Power data will be use in Power Company. It will clear how to analyze and evaluate the accuracy of data, detect electrical fraud, detect abnormal data if any... !

- Question 1 : How many meter in Tranformer Area ? How many meter is running (Active Meter )? How many meter of each model ?
- Question 2 : How about Success rate of each data that system can collect from meter as each day ? It need to show the chart to show the success rate of last 30 day ?
- Question 3 : Which each type of daily data. What is unormal data and what meter have unormal data ? Show the unormal data value if have ?

In [ ]:
# import pandas lib as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('mode.chained_assignment', None)

# Section 2: Data Understanding

In [ ]:
# Get Meter List and daily data 
MeterList = pd.read_excel('../Data/19193111_MeterList.xls')
DailyForwardEnergy = pd.read_excel('../Data/19193111_Daily_Forward_Energy.xls')
DailyReverseEnergy = pd.read_excel('../Data/19193111_Daily_Reverse_Energy.xls')
DailyVoltage = pd.read_excel('../Data/19193111_Daily_Voltage.xls')
DailyCurrent = pd.read_excel('../Data/19193111_Daily_Current.xls')

# Meter List is infomation of all meter in Tranformer Area. 
# It use to know about infomation and para of meter
# It include Meter No, status of meter to clear it is running or not (mean install on grid power or not), meter Model to clear meter type in this project we just focus about single phase meter and three phase meter. 
# Other information is Customer No and Customer Name.
print('\n- MeterList: print 02 top row ----------------')
print(MeterList.head(2))

# DailyForwardEnergy is Forward Energy value of meter that collect one time per day.
# It use to know how much power that customer already used. 
# it include Meter No, time collect data and Energy value of total and each tariff
print('\n- DailyForwardEnergy: print 02 top row ----------------')
print(DailyForwardEnergy.head(2))

# DailyReverseEnergy is Reverse Energy value of meter that collect one time per day. 
# It use to know how much power that customer return to power (with customer user solar power) or meter install wrong. 
# It include Meter No, time collect data and Energy value of total and each tariff
print('\n- DailyReverseEnergy: print 02 top row ----------------')
print(DailyReverseEnergy.head(2))

# DailyVoltage is Voltage value of meter that collect one time per day. 
# It use to know how the voltage value that is one of operating parameters of the grid power. 
# It include Meter No, time collect data and voltage value of each phase
print('\n- DailyVoltage: print 02 top row ----------------')
print(DailyVoltage.head(2))

# DailyCurrent is Current value of meter that collect one time per day. 
# It use to know the Current value that is one of operating parameters of the grid power. 
# It include Meter No, time collect data and Current value of each phase
print('\n- DailyCurrent: print 02 top row ----------------')
print(DailyCurrent.head(2))

# Section 3: Data Preparation

In [ ]:
# Normal we just handel data with Meter that is runnign on grid power. So we must get Meter have Status is Running (ActiveMeter)
ActiveMeterList = MeterList[(MeterList.Status == 'Running')]

# Get Meter Model : MM11-V1 is single phase meter, MM31/38 is Three phase meter
MeterModel = ActiveMeterList.Meter_Model.drop_duplicates()

# Get Single Phase Meter. With single phase meter it just inlcude a phase so it just have voltage and current value of phase L1
SinglePhaseMeterList = ActiveMeterList[(ActiveMeterList.Meter_Model == 'MM11-V1')]

# Get Three Phase Meter. With three phase meter it inlcude three phase so it will have voltage and current value of phase L1, Phase L2 and Phase L3
ThreePhaseMeterList = ActiveMeterList[(ActiveMeterList.Meter_Model == 'MM31/38')]

In [ ]:
# DEF function for Data Preparation
def DataPreparation(ActiveMeterList,DailyData):
    # Add column Flag Time By Day
    DailyData['Flag_Time_By_Day'] = DailyData.Data_Time.str.slice(0, 10)

    # Join Daily data with meter List by Meter_No
    ActiveMeterListWithDailyData=ActiveMeterList.set_index('Meter_No').join(DailyData.set_index('Meter_No'))

    # Get list Day had collected data from Active Meter List
    DataByDay = DailyData.Flag_Time_By_Day.drop_duplicates()
    
    # Get Success rate of Data by Day
    Output= pd.DataFrame()
    Output['Day'] = DataByDay.copy()
    Success_Rate=[]
    for item in Output['Day']:
        TempData = ActiveMeterListWithDailyData[ActiveMeterListWithDailyData.Flag_Time_By_Day==item]
        Success_Rate.append(round(TempData.shape[0]*100/ActiveMeterList.shape[0],2))
        
    Output['Success_Rate'] = Success_Rate
    
    return ActiveMeterListWithDailyData, Output

In [ ]:
# Active Meter List With Forward Energy Data And Success Rate Data (how many meter have rorward energy data in day per total active meter)
ActiveMeterListWithForwardEnergyData, SuccessRateForwardEnergyData = DataPreparation(ActiveMeterList,DailyForwardEnergy)

# Active Meter List With Reverse Energy Data And Success Rate Data (how many meter have reverse energy data in day per total active meter)
ActiveMeterListWithReverseEnergyData, SuccessRateReverseEnergyData = DataPreparation(ActiveMeterList,DailyReverseEnergy)

# Active Meter List With Voltage Data And Success Rate Data (how many meter have voltage data in day per total active meter)
ActiveMeterListWithVoltageData, SuccessRateVoltageData = DataPreparation(ActiveMeterList,DailyVoltage)

# Active Meter List With Current Data And Success Rate Data (how many meter have current data in day per total active meter)
ActiveMeterListWithCurrentData, SuccessRateCurrentData = DataPreparation(ActiveMeterList,DailyCurrent)

# Section 4: Evaluate the Results


- Question 1 : How many meter in Tranformer Area ? How many meter is running (Active Meter )? How many meter of each model ?

In [ ]:
# To check data. We must know how many active meter need to collect data
# Get Total Meter
TotalMeter = MeterList.shape[0]
print('\n- Total Meter: {total}'.format(total=TotalMeter))

# Get Meter have status is Active
TotalActiveMeter = ActiveMeterList.shape[0]
print('\n- Total Active Meter : {totalActive}'.format(totalActive=TotalActiveMeter))

# Get Meter Model : MM11-V1 is single phase meter, MM31/38 is Three phase meter
TotalSinglePhaseMeter = SinglePhaseMeterList.shape[0]
TotalThreePhaseMeter = ThreePhaseMeterList.shape[0]
print('\n- Total Sing Meter : {first}; Total Three Phase Meter : {second}'.format(first=TotalSinglePhaseMeter,second=TotalThreePhaseMeter))


- Question 2 : How about Success rate of each data that system can collect from meter as each day ? It need to show the chart to show the success rate of last 30 day ?

In [ ]:
# With daily data, we need to check the data collect from meter enough or not, check the data is normal or not. it is Success rate. DEF function for show Data Rate
def ShowDataRate(SuccessRateData,DataName,NumberDays): 
    # Show Success rate of last        
    print('\n- Success rate of daily {first} data of lase {second}:\n {third}'.format(first=DataName,second=NumberDays,third=SuccessRateData.head(NumberDays)))
    
    # Show Chart of Success_Rate by Day
    # create data
    SuccessRateData = SuccessRateData.sort_values(by=['Day'])
    x=SuccessRateData['Day'].head(NumberDays)
    y=SuccessRateData['Success_Rate'].head(NumberDays)

    # Change the style of plot
    plt.figure(figsize=(18, 9))
    plt.xticks(rotation = 90)
    plt.style.use('seaborn-darkgrid')
    
    # Make the same graph
    plt.fill_between( x, y, color="skyblue", alpha=0.3)
    plt.plot(x, y, color="skyblue")
    
    # Add title and axis names
    plt.title('Success rate of collect {first} data last {second} days'.format(first=DataName,second=NumberDays))
    plt.xlabel('Day')
    plt.ylabel('Success rate (%)')
    
    # Show graph
    plt.show()


## DAILY VOLTAGE DATA

Success Rate

Check Unomal Data

In [ ]:
# Voltage at Viet Nam normal from 176 V to 264 V. Check unormal data
max = 264
min = 176
UnormalVoltageData = ActiveMeterListWithVoltageData[((ActiveMeterListWithVoltageData.Voltage_in_phase_L1>max)|
                                                    (ActiveMeterListWithVoltageData.Voltage_in_phase_L1<min))|
                                                     ((ActiveMeterListWithVoltageData.Voltage_in_phase_L2>max)|
                                                    (ActiveMeterListWithVoltageData.Voltage_in_phase_L2<min))|
                                                     ((ActiveMeterListWithVoltageData.Voltage_in_phase_L3>max)|
                                                    (ActiveMeterListWithVoltageData.Voltage_in_phase_L3<min))]

TotalUnormalVoltageData = UnormalVoltageData.shape[0]
print('\n- Unormal Voltage Data :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalVoltageData,second=UnormalVoltageData))

# Three Phase meter normal must have voltage value of all Phase. Check unormal data. Just check with meter can collect data

VoltageDataThreePhaseMeter = ActiveMeterListWithVoltageData[ActiveMeterListWithVoltageData.Meter_Model=='MM31/38']
if VoltageDataThreePhaseMeter.shape[0]>0:
    VoltageDataThreePhaseMeter['CheckPhase1'] = VoltageDataThreePhaseMeter['Voltage_in_phase_L1'].isnull()
    VoltageDataThreePhaseMeter['CheckPhase2'] = VoltageDataThreePhaseMeter['Voltage_in_phase_L2'].isnull()
    VoltageDataThreePhaseMeter['CheckPhase3'] = VoltageDataThreePhaseMeter['Voltage_in_phase_L3'].isnull()

    UnormalVoltageDataThreePhaseMeter = VoltageDataThreePhaseMeter[((VoltageDataThreePhaseMeter.CheckPhase1==True)|
                                                                        (VoltageDataThreePhaseMeter.CheckPhase2==True)|
                                                                        (VoltageDataThreePhaseMeter.CheckPhase3==True))&
                                                                        ((VoltageDataThreePhaseMeter.CheckPhase1==False)|
                                                                        (VoltageDataThreePhaseMeter.CheckPhase2==False)|
                                                                        (VoltageDataThreePhaseMeter.CheckPhase3==False))]

    TotalUnormalVoltageDataThreePhaseMeter = UnormalVoltageDataThreePhaseMeter.shape[0]
    print('\n- Unormal Voltage Data Three Phase Meter :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalVoltageDataThreePhaseMeter,second=UnormalVoltageDataThreePhaseMeter))
else:
    print('\n- Do not have Current Data of Three Phase Meter')  

## DAILY CURRENT DATA

Success Rate

Check Unomal Data

In [ ]:
# Current range of single phase meter is 0 - 80A (Imax = 80A), Three phase meter is 0 - 100A (Imax = 100A). Meter can run normal if Current value < 120% Imax. Check Meter have unomal Current
ImaxSinglePhaseMeter = 80
ImaxThreePhaseMeter = 100

# Check Current range of single phase meter
CurrentDataSinglePhaseMeter = ActiveMeterListWithCurrentData[ActiveMeterListWithCurrentData.Meter_Model=='MM11-V1']

if CurrentDataSinglePhaseMeter.shape[0]>0:
    UnormalCurrentDataSinglePhaseMeter = CurrentDataSinglePhaseMeter[CurrentDataSinglePhaseMeter.Current_in_phase_L1>ImaxSinglePhaseMeter]

    TotalUnormalCurrentDataSinglePhaseMeter = UnormalCurrentDataSinglePhaseMeter.shape[0]
    print('\n- Unormal Current Data Single Phase Meter :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalCurrentDataSinglePhaseMeter,second=UnormalCurrentDataSinglePhaseMeter))
else:
    print('\n- Do not have Current Data of Single Phase Meter')    
# Check Current range of single phase meter

CurrentDataThreePhaseMeter = ActiveMeterListWithCurrentData[ActiveMeterListWithCurrentData.Meter_Model=='MM31/38']
if CurrentDataThreePhaseMeter.shape[0]>0:
    UnormalCurrentDataThreePhaseMeter = CurrentDataThreePhaseMeter[(CurrentDataThreePhaseMeter.Current_in_phase_L1>ImaxThreePhaseMeter)|
                                                                (CurrentDataThreePhaseMeter.Current_in_phase_L2>ImaxThreePhaseMeter)|
                                                                (CurrentDataThreePhaseMeter.Current_in_phase_L3>ImaxThreePhaseMeter)]

    TotalUnormalCurrentDataThreePhaseMeter = UnormalCurrentDataThreePhaseMeter.shape[0]
    print('\n- Unormal Current Data Three Phase Meter :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalCurrentDataThreePhaseMeter,second=UnormalCurrentDataThreePhaseMeter))
else:
    print('\n- Do not have Current Data of Three Phase Meter')    

## DAILY FORWARD ENERGY DATA

Success Rate

In [ ]:
# Success Rate (how many meter have this data in day per total active meter)
ActiveMeterListWithForwardEnergyData, SuccessRateForwardEnergyData = DataPreparation(ActiveMeterList,DailyForwardEnergy,'Forward Energy')

Check Unomal Data

In [ ]:
# Normal Data of Enegry is the value >=0. Check that
UnormalEnegryDataCheckValue = ActiveMeterListWithForwardEnergyData[(ActiveMeterListWithForwardEnergyData.Active_energy_total < 0) |
                                 (ActiveMeterListWithForwardEnergyData.Active_energy_tariff1 < 0) | 
                                 (ActiveMeterListWithForwardEnergyData.Active_energy_tariff2 < 0) | 
                                 (ActiveMeterListWithForwardEnergyData.Active_energy_tariff3 < 0) | 
                                 (ActiveMeterListWithForwardEnergyData.Active_energy_tariff4 < 0) |
                                 (ActiveMeterListWithForwardEnergyData.Reactive_energy_total < 0) |
                                 (ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff1 < 0) | 
                                 (ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff2 < 0) | 
                                 (ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff3 < 0) | 
                                 (ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff4 < 0)]

TotalUnormalEnegryDataCheckValue = UnormalEnegryDataCheckValue.shape[0]
print('\n- Unormal Enegery (Enegry value <0) :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalEnegryDataCheckValue,second=UnormalEnegryDataCheckValue))


# With multi tariff meter TotalEnegry = (99% - 101%) of (EnegryTariff1 + EnegryTariff2 + EnegryTariff3 + EnegryTariff4). Check that
minvalue = 0.99
maxvalue = 1.01
UnormalEnegryDataCheckTotalAndTariffValue = ActiveMeterListWithForwardEnergyData[((ActiveMeterListWithForwardEnergyData.Active_energy_total < (ActiveMeterListWithForwardEnergyData.Active_energy_tariff1 + ActiveMeterListWithForwardEnergyData.Active_energy_tariff2 + ActiveMeterListWithForwardEnergyData.Active_energy_tariff3 + ActiveMeterListWithForwardEnergyData.Active_energy_tariff4)*minvalue)|
                                               (ActiveMeterListWithForwardEnergyData.Active_energy_total > (ActiveMeterListWithForwardEnergyData.Active_energy_tariff1 + ActiveMeterListWithForwardEnergyData.Active_energy_tariff2 + ActiveMeterListWithForwardEnergyData.Active_energy_tariff3 + ActiveMeterListWithForwardEnergyData.Active_energy_tariff4)*maxvalue))|
                                               ((ActiveMeterListWithForwardEnergyData.Reactive_energy_total < (ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff1 + ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff2 + ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff3 + ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff4)*minvalue)|
                                               (ActiveMeterListWithForwardEnergyData.Reactive_energy_total > (ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff1 + ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff2 + ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff3 + ActiveMeterListWithForwardEnergyData.Reactive_energy_tariff4)*maxvalue))]

TotalUnormalEnegryDataCheckTotalAndTariffValue = UnormalEnegryDataCheckTotalAndTariffValue.shape[0]
print('\n- With multi tariff meter, Unormal Enegery (TotalEnegry not same with (EnegryTariff1 + EnegryTariff2 + EnegryTariff3 + EnegryTariff4)) :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalEnegryDataCheckTotalAndTariffValue,second=UnormalEnegryDataCheckTotalAndTariffValue))

## DAILY REVERSE ENERGY DATA

Success Rate

In [ ]:
# Success Rate (how many meter have this data in day per total active meter)
ActiveMeterListWithReverseEnergyData, SuccessRateReverseEnergyData = getDataRate(ActiveMeterList,DailyReverseEnergy,'Reverse Energy')

Check Unomal Data

In [ ]:
# Normal Data of Enegry is the value >=0. Check that
UnormalEnegryDataCheckValue = ActiveMeterListWithReverseEnergyData[(ActiveMeterListWithReverseEnergyData.Active_energy_total < 0) |
                                 (ActiveMeterListWithReverseEnergyData.Active_energy_tariff1 < 0) | 
                                 (ActiveMeterListWithReverseEnergyData.Active_energy_tariff2 < 0) | 
                                 (ActiveMeterListWithReverseEnergyData.Active_energy_tariff3 < 0) | 
                                 (ActiveMeterListWithReverseEnergyData.Active_energy_tariff4 < 0) |
                                 (ActiveMeterListWithReverseEnergyData.Reactive_energy_total < 0) |
                                 (ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff1 < 0) | 
                                 (ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff2 < 0) | 
                                 (ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff3 < 0) | 
                                 (ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff4 < 0)]

TotalUnormalEnegryDataCheckValue = UnormalEnegryDataCheckValue.shape[0]
print('\n- Unormal Enegery (Enegry value <0) :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalEnegryDataCheckValue,second=UnormalEnegryDataCheckValue))


# With multi tariff meter TotalEnegry = (99% - 101%) of (EnegryTariff1 + EnegryTariff2 + EnegryTariff3 + EnegryTariff4). Check that
minvalue = 0.99
maxvalue = 1.01
UnormalEnegryDataCheckTotalAndTariffValue = ActiveMeterListWithReverseEnergyData[((ActiveMeterListWithReverseEnergyData.Active_energy_total < (ActiveMeterListWithReverseEnergyData.Active_energy_tariff1 + ActiveMeterListWithReverseEnergyData.Active_energy_tariff2 + ActiveMeterListWithReverseEnergyData.Active_energy_tariff3 + ActiveMeterListWithReverseEnergyData.Active_energy_tariff4)*minvalue)|
                                               (ActiveMeterListWithReverseEnergyData.Active_energy_total > (ActiveMeterListWithReverseEnergyData.Active_energy_tariff1 + ActiveMeterListWithReverseEnergyData.Active_energy_tariff2 + ActiveMeterListWithReverseEnergyData.Active_energy_tariff3 + ActiveMeterListWithReverseEnergyData.Active_energy_tariff4)*maxvalue))|
                                               ((ActiveMeterListWithReverseEnergyData.Reactive_energy_total < (ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff1 + ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff2 + ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff3 + ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff4)*minvalue)|
                                               (ActiveMeterListWithReverseEnergyData.Reactive_energy_total > (ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff1 + ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff2 + ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff3 + ActiveMeterListWithReverseEnergyData.Reactive_energy_tariff4)*maxvalue))]

TotalUnormalEnegryDataCheckTotalAndTariffValue = UnormalEnegryDataCheckTotalAndTariffValue.shape[0]
print('\n- With multi tariff meter, Unormal Enegery (TotalEnegry not same with (EnegryTariff1 + EnegryTariff2 + EnegryTariff3 + EnegryTariff4)) :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalEnegryDataCheckTotalAndTariffValue,second=UnormalEnegryDataCheckTotalAndTariffValue))
